In [1]:
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import eli5
from eli5.sklearn import PermutationImportance

%matplotlib inline

In [2]:
train = pd.read_csv('df_train.csv') # dummified dataset
test = pd.read_csv('df_test.csv')

In [3]:
train.head()

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,A1Cresult_0,A1Cresult_1,diag_1_1.0,diag_1_2.0,diag_1_3.0,diag_1_4.0,diag_1_5.0,diag_1_6.0,diag_1_7.0,diag_1_8.0
0,149190,55629189,2,-0.438603,0.805968,-0.809310,0.277594,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,0,0,0,0,0
1,500364,82442376,4,-0.777582,0.051157,-0.238557,0.036471,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,0,0,0,0,0
2,16680,42519267,5,-1.116561,0.403402,-0.809310,-0.928017,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,0,0,0,0,1
3,35754,82637451,6,-0.438603,-0.603012,2.615208,0.036471,-0.264763,-0.205982,-0.329314,...,0,0,1,0,0,0,0,0,0,0
4,55842,84259809,7,-0.099625,1.359497,-0.238557,0.639277,-0.264763,-0.205982,-0.329314,...,0,0,1,0,0,0,0,0,0,0


In [4]:
test.head()

,encounter_id,patient_nbr,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,...,A1Cresult_0,A1Cresult_1,diag_1_1.0,diag_1_2.0,diag_1_3.0,diag_1_4.0,diag_1_5.0,diag_1_6.0,diag_1_7.0,diag_1_8.0
0,64410,86047875,3,-0.777582,-1.609427,2.044455,-0.325212,1.592509,-0.205982,1.224358,...,0,0,0,0,0,0,0,0,0,0
1,12522,48330783,9,2.951182,1.258855,0.332196,1.483204,-0.264763,-0.205982,-0.329314,...,0,0,1,0,0,0,0,0,0,0
2,28236,89869032,5,1.595268,0.202119,0.332196,0.157032,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,1,0,0,0,0
3,216156,62718876,8,-0.438603,-1.206861,1.473702,0.277594,-0.264763,-0.205982,-0.329314,...,0,0,0,0,0,0,0,0,0,1
4,253722,96664626,8,-1.116561,0.504044,-0.809310,-0.686895,-0.264763,-0.205982,-0.329314,...,0,0,0,1,0,0,0,0,0,0


In [5]:
# Create X and y for train and test sets 
X_train = train.drop('readmitted', axis=1)
y_train = train['readmitted']

X_test = test.drop('readmitted', axis=1)
y_test = test['readmitted']

In [6]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [7]:
# machine learning
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import skew
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, precision_score, recall_score, roc_auc_score
import warnings
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from boruta import BorutaPy
from xgboost import XGBClassifier, XGBRanker
print(__doc__)

# Definitions
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
warnings.filterwarnings('ignore')

Automatically created module for IPython interactive environment


In [8]:
classifiers = [KNeighborsClassifier(3),
#     SVC(kernel="rbf", C=0.025, probability=True),
#     NuSVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()
]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Precision", "Recall", "AUC", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    pre = precision_score(y_test, train_predictions)
    rec = recall_score(y_test, train_predictions)
    AUC = roc_auc_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    print("Precision: {:.4%}".format(pre))
    print("Recall: {:.4%}".format(rec))
    print("AUC: {}".format(AUC))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, pre*100, rec*100, AUC, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 93.5084%
Precision: 4.3478%
Recall: 1.9523%
AUC: 0.4993970666782939
Log Loss: 1.3406216567513574
DecisionTreeClassifier
****Results****
Accuracy: 86.2978%
Precision: 6.7426%
Recall: 15.4013%
AUC: 0.5256036600210016
Log Loss: 4.732567783263503
RandomForestClassifier
****Results****
Accuracy: 95.3061%
Precision: 0.0000%
Recall: 0.0000%
AUC: 0.4995288944723618
Log Loss: 0.6898753976840869
AdaBoostClassifier
****Results****
Accuracy: 95.3960%
Precision: 0.0000%
Recall: 0.0000%
AUC: 0.5
Log Loss: 0.6696045950947239
GradientBoostingClassifier
****Results****
Accuracy: 95.3960%
Precision: 50.0000%
Recall: 0.2169%
AUC: 0.501032253639855
Log Loss: 0.18545875949550272
GaussianNB
****Results****
Accuracy: 95.3960%
Precision: 0.0000%
Recall: 0.0000%
AUC: 0.5
Log Loss: 0.19649680550680965
LinearDiscriminantAnalysis
****Results****
Accuracy: 95.3660%
Precision: 40.0000%
Recall: 1.3015%
AUC: 0.5060364866632512
Log Loss: 0.18586142387915694
QuadraticDiscr

In [9]:
log

,Classifier,Accuracy,Precision,Recall,AUC,Log Loss
0,KNeighborsClassifier,93.508,4.348,1.952,0.499,1.341
0,DecisionTreeClassifier,86.298,6.743,15.401,0.526,4.733
0,RandomForestClassifier,95.306,0.000,0.000,0.500,0.690
0,AdaBoostClassifier,95.396,0.000,0.000,0.500,0.670
0,GradientBoostingClassifier,95.396,50.000,0.217,0.501,0.185
0,GaussianNB,95.396,0.000,0.000,0.500,0.196
0,LinearDiscriminantAnalysis,95.366,40.000,1.302,0.506,0.186
0,QuadraticDiscriminantAnalysis,95.396,0.000,0.000,0.500,1.590


In [10]:
sgd = SGDClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
dtc = DecisionTreeClassifier()
abc = AdaBoostClassifier()
xgb = XGBClassifier()

In [12]:
model=abc.fit(X_train, y_train)
perm = PermutationImportance(abc, random_state=1).fit(X_train, y_train)

In [13]:
eli5.show_weights(perm, top=None)

Weight,Feature
0.0001 ± 0.0001,x61
0.0000 ± 0.0001,x66
0.0000 ± 0.0000,x4
0.0000 ± 0.0000,x46
0.0000 ± 0.0000,x60
0.0000 ± 0.0001,x59
0.0000 ± 0.0000,x0
0 ± 0.0000,x33
0 ± 0.0000,x32
0 ± 0.0000,x31


In [ ]:
time_in_hospital, num_procedures, number_emergency, admission_type_id

In [16]:
model3 = rfc.fit(X_train, y_train)
perm3 = PermutationImportance(rfc, random_state=1).fit(X_train, y_train)
eli5.show_weights(perm3, top=None)

Weight,Feature
0.0334 ± 0.0004,x46
0.0318 ± 0.0013,x0
0.0294 ± 0.0003,x45
0.0293 ± 0.0009,x47
0.0293 ± 0.0005,x49
0.0288 ± 0.0007,x1
0.0276 ± 0.0007,x42
0.0264 ± 0.0004,x3
0.0254 ± 0.0005,x44
0.0251 ± 0.0007,x59


In [17]:
eli5.show_weights(perm3, top=None, show='feature_importance')

#### Explore using BorutaPy to Select and Rank Features

In [18]:
# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
X = X_train.values
y = y_train.values
y = y.ravel()

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

# find all relevant features - 5 features should be selected
feat_selector.fit(X, y)

# check selected features - first 5 features are selected
feat_selector.support_

# check ranking of features
feat_selector.ranking_

# call transform() on X to filter it down to selected features
X_filtered = feat_selector.transform(X)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	84
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	84
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	84
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	84
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	84
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	84
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	84
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	16
Tentative: 	0
Rejected: 	68


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	16
Tentative: 	0
Rejected: 	68


In [19]:
feat_selector.support_

array([ True,  True,  True,  True,  True, False,  True, False, False,
        True, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False,  True, False,  True,
        True,  True,  True, False,  True, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [20]:
feat_selector.ranking_

array([ 1,  1,  1,  1,  1, 10,  1, 21, 16,  1, 11,  9,  1, 20, 40, 49, 53,
       43, 63, 29, 38, 55, 40, 36, 52, 55, 63, 55, 12, 50, 63, 63, 63, 63,
       25,  7, 63, 13,  2,  1, 14, 18,  1,  3,  1,  1,  1,  1,  4,  1,  6,
       51, 28, 46, 47, 29, 18, 55, 37,  1, 44, 22, 47, 63, 63, 63, 15, 35,
       27, 63, 39, 63, 45, 42, 33, 25,  4,  8, 31, 18, 34, 23, 32, 24])

In [21]:
X_filtered = feat_selector.transform(X)

In [22]:
X_filtered

array([[1.49190000e+05, 5.56291890e+07, 2.00000000e+00, ...,
        1.80000000e+01, 2.70000000e+01, 0.00000000e+00],
       [5.00364000e+05, 8.24423760e+07, 4.00000000e+00, ...,
        2.80000000e+01, 1.40000000e+01, 0.00000000e+00],
       [1.66800000e+04, 4.25192670e+07, 5.00000000e+00, ...,
        2.50000000e+01, 5.00000000e+00, 0.00000000e+00],
       ...,
       [4.43842070e+08, 1.40199494e+08, 7.00000000e+00, ...,
        6.30000000e+01, 1.80000000e+01, 0.00000000e+00],
       [4.43847548e+08, 1.00162476e+08, 8.00000000e+00, ...,
        7.20000000e+01, 2.70000000e+01, 1.00000000e+00],
       [4.43867222e+08, 1.75429310e+08, 8.00000000e+00, ...,
        7.20000000e+01, 5.40000000e+01, 0.00000000e+00]])

In [23]:
print ('\n Initial features: ', X_train.columns.tolist() )

# number of selected features
print ('\n Number of selected features:')
print (feat_selector.n_features_)

feature_df = pd.DataFrame(X_train.columns.tolist(), columns=['features'])
feature_df['rank']=feat_selector.ranking_
feature_df = feature_df.sort_values('rank', ascending=True).reset_index(drop=True)
print ('\n Top %d features:' % feat_selector.n_features_)
print (feature_df.head(feat_selector.n_features_))
# feature_df.to_csv('boruta-feature-ranking.csv', index=False)

# check ranking of features
print ('\n Feature ranking:')
print (feat_selector.ranking_)



 Initial features:  ['encounter_id', 'patient_nbr', 'age', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_2', 'diag_3', 'number_diagnoses', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin', 'glyburide.metformin', 'glipizide.metformin', 'glimepiride.pioglitazone', 'metformin.rosiglitazone', 'metformin.pioglitazone', 'change', 'diabetesMed', 'IsTrain', 'numchange', 'num_meds', 'number_inpatient_log1p', 'number_emergency_log1p', 'number_outpatient_log1p', 'num_medications|time_in_hospital', 'num_medications|num_procedures', 'time_in_hospital|num_lab_procedures', 'num_medications|num_lab_procedures', 'num_medications|number_diagnoses', 'age|number_diagnoses', 'change|num_medications', 'number_diagnoses|time_in_ho

#### Use Synthetic Minority Over-Sampling Technique to balance train data set (90:10 -> 50:50)

In [24]:
from imblearn.over_sampling import SMOTE
from collections import Counter
print('Original dataset shape {}'.format(Counter(y_train)))
sm = SMOTE(random_state=42)
X_train_new, y_train_new = sm.fit_sample(X_train, y_train)
print('New dataset shape {}'.format(Counter(y_train_new)))

Original dataset shape Counter({0: 47345, 1: 4414})
New dataset shape Counter({0: 47345, 1: 47345})


In [25]:
classifiers = [KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()
]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Precision", "Recall", "AUC", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train_new, y_train_new)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    pre = precision_score(y_test, train_predictions)
    rec = recall_score(y_test, train_predictions)
    AUC = roc_auc_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    print("Precision: {:.4%}".format(pre))
    print("Recall: {:.4%}".format(rec))
    print("AUC: {}".format(AUC))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, pre*100, rec*100, AUC, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

KNeighborsClassifier
****Results****
Accuracy: 68.9604%
Precision: 4.9677%
Recall: 31.6703%
AUC: 0.5121516612345893
Log Loss: 5.044567709364535
DecisionTreeClassifier
****Results****
Accuracy: 86.3777%
Precision: 7.2038%
Recall: 16.4859%
AUC: 0.5311836886892888
Log Loss: 4.704972635839226
RandomForestClassifier
****Results****
Accuracy: 95.2562%
Precision: 11.1111%
Recall: 0.4338%
AUC: 0.5013316764589396
Log Loss: 0.5855890658363907
AdaBoostClassifier
****Results****
Accuracy: 95.3960%
Precision: 0.0000%
Recall: 0.0000%
AUC: 0.5
Log Loss: 0.6745291966391999
GradientBoostingClassifier
****Results****
Accuracy: 95.3960%
Precision: 0.0000%
Recall: 0.0000%
AUC: 0.5
Log Loss: 0.2092390420763025
GaussianNB
****Results****
Accuracy: 30.4504%
Precision: 5.2936%
Recall: 83.5141%
AUC: 0.5570177350963059
Log Loss: 0.6862719215606566
LinearDiscriminantAnalysis
****Results****
Accuracy: 66.6134%
Precision: 8.3526%
Recall: 62.6898%
AUC: 0.6474628429566487
Log Loss: 0.6441286606120061
QuadraticDiscri

In [26]:
log

,Classifier,Accuracy,Precision,Recall,AUC,Log Loss
0,KNeighborsClassifier,68.960,4.968,31.670,0.512,5.045
0,DecisionTreeClassifier,86.378,7.204,16.486,0.531,4.705
0,RandomForestClassifier,95.256,11.111,0.434,0.501,0.586
0,AdaBoostClassifier,95.396,0.000,0.000,0.500,0.675
0,GradientBoostingClassifier,95.396,0.000,0.000,0.500,0.209
0,GaussianNB,30.450,5.294,83.514,0.557,0.686
0,LinearDiscriminantAnalysis,66.613,8.353,62.690,0.647,0.644
0,QuadraticDiscriminantAnalysis,95.396,0.000,0.000,0.500,1.590
